<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-01 11:29:02
-------------------
qualified stocks: 95
with latest results: 30
still star stocks: 20
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.03 L
Current:  1.37 C
-------------------
Today PnL: -6.70 K (-0.05%)
Current PnL: -17.73 L (-12.06%)
CY Booked + Current PnL: -9.70 L (-6.6%)
-------------------
Total profit:  3.20 L
Total loss:  -20.93 L
-------------------
Total Booked + Current PnL: 17.28 L (14.4%)
Total Booked PnL: 35.01 L (29.16%)
Curr Year Booked PnL: 8.03 L (5.87%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 76.41 L (55.81%)
Deployed:  1.20 C
Current:  1.37 C
CAGR/XIRR %: 8.68%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,HINDALCO,651.95,761.55,-21.55,H-LC,5.24,108192.0,3880.0,13654.0,-1.00,3.72,12.62,16.81,10.0,0.28,0.80,20.36,X5K,ATH,METALS
74,SIS,477.00,477.00,2364.25,H-SC,6.88,66300.0,-14790.0,14792.0,0.93,-18.24,22.31,0.00,157.0,-1.00,0.49,34.37,OX40N,NTT,MISC
29,DIXON,14367.00,18940.15,-26.03,H-MC,6.14,136192.0,21256.0,15335.0,1.09,18.49,11.26,31.83,85.0,1.39,1.01,53.27,X40N,ATH,IT
70,SBIN,760.30,863.00,-12.46,M-LC,4.19,211934.0,10454.0,16764.0,0.40,5.19,7.91,13.51,62.0,0.62,1.57,18.44,XY25,NTT,BANKS
80,TATAELXSI,7332.28,7332.00,-18.99,H-MC,1.91,84959.0,-17693.0,17688.0,-0.41,-17.24,20.82,-0.00,101.0,-1.00,0.63,29.15,OX40N,NTT,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,QUESS,296.83,986.00,-34.99,M-SC,41.77,64397.0,-609.0,151539.0,-0.19,-0.94,235.32,232.18,199.0,-0.00,0.48,0.05,XY24,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,8.94,H-SC,19.95,101338.0,375.0,94153.0,0.36,0.37,92.91,93.63,149.0,0.00,0.75,50.94,SR,ATH,CHEMICALS
20,CAMS,3643.00,5226.82,-8.36,H-SC,0.81,102878.0,874.0,43476.0,-1.67,0.86,42.26,43.48,120.0,0.02,0.76,17.78,X40N,ATH,MISC
38,HEROMOTOCO,4311.81,5949.07,-6.67,H-MC,3.65,150276.0,-637.0,57946.0,0.77,-0.42,38.56,37.97,90.0,-0.01,1.11,24.78,AR,ATH,AUTO
94,WIPRO,243.46,420.00,-11.94,M-LC,5.19,151168.0,223.0,109234.0,-1.80,0.15,72.26,72.51,55.0,0.00,1.12,6.18,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.0,-21.14,H-LC,1.74,207486.0,2986.0,38426.0,-2.58,1.46,18.52,20.25,9.0,0.08,1.54,10.32,X40N,BTT,PHARMA
56,LTIM,5564.16,7230.2,-5.37,H-LC,2.82,182124.0,-18186.0,78168.0,-0.92,-9.08,42.92,29.94,24.0,-0.23,1.35,25.82,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-10.42,H-LC,2.86,185878.0,-14628.0,63403.0,-0.47,-7.30,34.11,24.33,30.0,-0.23,1.38,7.19,X40N,BTT,FINANCE
5,ASIANPAINT,2961.56,4250.0,-15.72,H-LC,3.17,167498.0,-33888.0,121503.0,2.80,-16.83,72.54,43.51,19.0,-0.28,1.24,16.60,X40,BTT,PAINTS
60,NESTLEIND,2268.60,2755.0,-16.28,H-LC,4.50,265648.0,222.0,56689.0,1.01,0.08,21.34,21.44,26.0,0.00,1.97,6.73,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,7.49,19916.0,-29677.0,878.0,3.84,-59.84,4.41,-58.07,261.0,-33.80,0.15,33.56,XY24,ATH,MISC
27,DEN,52.79,75.00,117.62,M-SC,3.32,61376.0,-23088.0,58626.0,3.65,-27.33,95.52,42.07,238.0,-0.39,0.46,30.34,AR,NTT,ENTERTAINMENT
5,ASIANPAINT,2961.56,4250.00,-15.72,H-LC,3.17,167498.0,-33888.0,121503.0,2.80,-16.83,72.54,43.51,19.0,-0.28,1.24,16.60,X40,BTT,PAINTS
18,BSOFT,628.85,836.99,-16.70,H-SC,12.68,70052.0,-39997.0,76420.0,2.25,-36.34,109.09,33.10,129.0,-0.52,0.52,16.40,XR,ATH,IT
53,KPIGREEN,497.21,731.64,29.10,H-SC,12.86,132111.0,6814.0,52263.0,2.24,5.44,39.56,47.15,144.0,0.13,0.98,65.28,MH,ATH,POWER


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.00,-21.14,H-LC,1.74,207486.0,2986.0,38426.0,-2.58,1.46,18.52,20.25,9.0,0.08,1.54,10.32,X40N,BTT,PHARMA
81,TATAMOTORS,761.77,1065.00,-45.88,M-LC,4.80,236494.0,-40029.0,150103.0,-2.17,-14.48,63.47,39.81,54.0,-0.27,1.75,13.34,XY24,NTT,AUTO
59,NATIONALUM,189.63,247.44,-52.42,H-MC,3.01,96142.0,-4362.0,35005.0,-1.98,-4.34,36.41,30.49,81.0,-0.12,0.71,26.66,MH,ATH,METALS
28,DIGITIDE,188.38,267.99,NaN,NaN,4.11,50937.0,9682.0,7753.0,-1.97,23.47,15.22,42.26,198.0,1.25,0.38,19.92,XY24,ATH,IT
94,WIPRO,243.46,420.00,-11.94,M-LC,5.19,151168.0,223.0,109234.0,-1.80,0.15,72.26,72.51,55.0,0.00,1.12,6.18,XR,NTT,IT


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,SYMPHONY,1306.42,1306.0,-30.89,M-SC,8.85,147296.0,-23845.0,23788.0,0.85,-13.93,16.15,-0.03,197.0,-1.00,1.09,6.59,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,128.24,H-SC,5.79,149519.0,-25040.0,25074.0,0.48,-14.34,16.77,0.02,135.0,-1.00,1.11,29.66,OX40N,NTT,PAINTS
86,UJJIVANSFB,52.77,53.0,36.52,H-SC,20.83,118746.0,-23733.0,24355.0,-0.66,-16.66,20.51,0.44,163.0,-0.97,0.88,40.15,OX40N,NTT,BANKS
80,TATAELXSI,7332.28,7332.0,-18.99,H-MC,1.91,84959.0,-17693.0,17688.0,-0.41,-17.24,20.82,-0.00,101.0,-1.00,0.63,29.15,OX40N,NTT,IT
52,KANSAINER,299.63,340.0,-68.02,H-SC,1.70,222759.0,-46908.0,83245.0,1.07,-17.39,37.37,13.47,136.0,-0.56,1.65,12.67,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,TCS,3794.03,4998.00,-27.70,H-LC,13.85,235576.0,-60358.0,154279.0,-0.55,-20.40,65.49,31.73,1.0,-0.39,1.75,0.00,X40,BTT,IT
32,ENRIN,1377.95,3237.90,NaN,NaN,3.19,80382.0,45933.0,563.0,-0.70,133.34,0.70,134.98,2.0,81.59,0.60,22.92,AR,ATH,ELECTRICAL
91,VBL,492.64,671.64,-8.37,H-LC,11.73,278092.0,13052.0,83261.0,-1.00,4.92,29.94,36.33,3.0,0.16,2.06,18.88,X40N,ATH,FMCG
48,INFY,1461.46,2275.00,-15.65,H-LC,7.13,270111.0,8510.0,137108.0,-0.66,3.25,50.76,55.67,4.0,0.06,2.00,9.56,X40,BTT,IT
1,ABB,5551.76,7934.00,-39.67,H-LC,7.55,215280.0,-1239.0,94142.0,0.18,-0.57,43.73,42.91,5.0,-0.01,1.60,12.53,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,4.11,50937.0,9682.0,7753.0,-1.97,23.47,15.22,42.26,198.0,1.25,0.38,19.92,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,20.18,H-SC,12.50,153932.0,27732.0,18441.0,1.21,21.97,11.98,36.59,156.0,1.50,1.14,37.24,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3237.90,NaN,NaN,3.19,80382.0,45933.0,563.0,-0.70,133.34,0.70,134.98,2.0,81.59,0.60,22.92,AR,ATH,ELECTRICAL


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,KPIGREEN,497.21,731.64,29.10,H-SC,12.86,132111.0,6814.0,52263.0,2.24,5.44,39.56,47.15,144.0,0.13,0.98,65.28,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,20.18,H-SC,12.50,153932.0,27732.0,18441.0,1.21,21.97,11.98,36.59,156.0,1.50,1.14,37.24,XY24,NTT,HEALTHCARE
3,ALKYLAMINE,2347.98,4546.37,8.94,H-SC,19.95,101338.0,375.0,94153.0,0.36,0.37,92.91,93.63,149.0,0.00,0.75,50.94,SR,ATH,CHEMICALS
26,DABUR,505.20,735.00,-3.23,H-MC,5.16,209742.0,12714.0,76912.0,1.66,6.45,36.67,45.49,100.0,0.17,1.56,20.33,XY24,BTT,FMCG
15,BLUESTARCO,1646.70,2326.38,-7.57,H-SC,9.48,173370.0,8700.0,59275.0,-0.25,5.28,34.19,41.28,119.0,0.15,1.29,13.69,X40N,ATH,AC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3237.90,NaN,NaN,3.19,80382.0,45933.0,563.0,-0.70,133.34,0.70,134.98,2.0,81.59,0.60,22.92,AR,ATH,ELECTRICAL
53,KPIGREEN,497.21,731.64,29.10,H-SC,12.86,132111.0,6814.0,52263.0,2.24,5.44,39.56,47.15,144.0,0.13,0.98,65.28,MH,ATH,POWER
45,INDIAMART,2327.09,4850.92,-47.14,H-SC,13.36,137471.0,14135.0,119627.0,0.18,11.46,87.02,108.45,118.0,0.12,1.02,36.38,AR,ATH,MISC
93,WHIRLPOOL,1219.98,2270.00,-40.95,M-SC,7.05,99382.0,7884.0,70869.0,-0.71,8.62,71.31,86.07,211.0,0.11,0.74,43.77,XR,NTT,DURABLES
94,WIPRO,243.46,420.00,-11.94,M-LC,5.19,151168.0,223.0,109234.0,-1.80,0.15,72.26,72.51,55.0,0.00,1.12,6.18,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,7.49,19916.0,-29677.0,878.0,3.84,-59.84,4.41,-58.07,261.0,-33.80,0.15,33.56,XY24,ATH,MISC
74,SIS,477.00,477.00,2364.25,H-SC,6.88,66300.0,-14790.0,14792.0,0.93,-18.24,22.31,0.00,157.0,-1.00,0.49,34.37,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,21.24,69408.0,-44141.0,84192.0,1.12,-38.87,121.30,35.27,267.0,-0.52,0.51,69.88,XR,NTT,HOTELS
32,ENRIN,1377.95,3237.90,NaN,NaN,3.19,80382.0,45933.0,563.0,-0.70,133.34,0.70,134.98,2.0,81.59,0.60,22.92,AR,ATH,ELECTRICAL
57,MASFIN,326.60,399.50,-19.60,H-SC,16.72,91350.0,-6630.0,28501.0,-1.15,-6.77,31.20,22.32,151.0,-0.23,0.68,32.49,XR,ATH,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,7.49,19916.0,-29677.0,878.0,3.84,-59.84,4.41,-58.07,261.0,-33.80,0.15,33.56,XY24,ATH,MISC
28,DIGITIDE,188.38,267.99,NaN,NaN,4.11,50937.0,9682.0,7753.0,-1.97,23.47,15.22,42.26,198.0,1.25,0.38,19.92,XY24,ATH,IT
29,DIXON,14367.00,18940.15,-26.03,H-MC,6.14,136192.0,21256.0,15335.0,1.09,18.49,11.26,31.83,85.0,1.39,1.01,53.27,X40N,ATH,IT
58,MEDANTA,1087.93,1486.00,20.18,H-SC,12.50,153932.0,27732.0,18441.0,1.21,21.97,11.98,36.59,156.0,1.50,1.14,37.24,XY24,NTT,HEALTHCARE
72,SHALBY,261.39,327.00,979.74,M-SC,8.96,146750.0,-34393.0,79861.0,1.01,-18.99,54.42,25.10,233.0,-0.43,1.09,17.61,XY24,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.48
1,25,41.15
2,50,70.49


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.40
LC    36.69
MC    21.77
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.26
X40N     14.97
X40      13.06
XY25     11.59
XR        8.85
AR        8.18
OX40N     7.71
X5K       2.15
MH        1.69
X200      1.35
SR        1.18
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.78
H-SC    26.96
H-MC    18.90
M-SC    12.01
M-LC     6.89
M-MC     2.54
L-SC     1.43
L-LC     1.02
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.27,-0.13,32.46
IT,12.19,-15.90,72.04
FINANCE,8.27,-16.98,59.42
BANKS,7.31,-15.48,62.60
PAINTS,5.73,-12.95,35.21
ELECTRICAL,5.34,-2.40,51.14
MISC,5.25,-22.40,85.85
HEALTHCARE,5.03,3.14,21.31
AUTO,4.22,-18.60,70.20


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2773592.0,25
XR,1098925.0,13
AR,1044557.0,10
X40,808890.0,10
X40N,589796.0,12
XY25,490945.0,8
OX40N,335885.0,10
SR,183509.0,2
X5K,149040.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2685159.0,30
M-SC,1352756.0,15
H-LC,1276609.0,20
H-MC,1154720.0,15
M-LC,448989.0,5
M-MC,335437.0,2
L-SC,268940.0,3
L-MC,57494.0,1
L-LC,51277.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,926295.0,7
M-SC,XY24,832838.0,7
H-SC,AR,610996.0,4
H-LC,X40,537505.0,6
H-SC,XR,513565.0,6
H-MC,XY24,443525.0,4
M-MC,XY24,335437.0,2
H-LC,X40N,293343.0,6
H-SC,X40N,222343.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
